### Step 1: Pseudocode
[LaTeX pseudo located here](https://github.com/NeuroDataDesign/orange-panda/blob/master/notes/bad_chan_detect/baddetec/bad-electrode-detection.pdf). ["English" pseudo located here (this is before I knew how to use LaTeX](https://github.com/NeuroDataDesign/orange-panda/blob/master/notes/bad_chan_detect/baddetec/pseudo.md)

Also, will need for implementation of kernel probability density funcitons:
[KDE Python Notebook (code.md approved)](https://github.com/NeuroDataDesign/orange-panda/blob/master/notes/bad_chan_detect/Kernel%20Density%20Estimates.ipynb)

***

### Step 2: Simulations & Details of Their Parameters

**For all sine waves created:** make sure number of points = 1000 sampled from each wave representing data.

***Success:***
- 5 sine waves, all 5 the same wave
    - Should result in none selected
- 5 sine waves, all 5 the same wave, 1 with lots of white noise
    - Should result in white noise one selected
- 5 sine waves, all same frequency but different amplitudes, 1 with white noise
    - Should result in white noise one selected

***Hopeful:***
- 10 sine waves, 8 with slight noise, 2 with heavy noise **CLOSEST TO REAL DATA**

***Fail:***
- 5 sine waves, all same frequency but different amplitudes, 4 with white noise
    - No idea result; either none selected or multiple



***

### Step 3: Choose Visualization
1. Will use connected scatter plots to show the simulated data (sine waves).
2. Then, after finding the joint probabilities across each of the sine waves (i.e. the electrode), will plot the joint probabilities as a distribution via KDE
3. Finally, comparing the number of standard of deviations from the mean to a threshold, find out if data remains or not. Print which waves stayed, and plot them (not sure if really necessary though).


***

### Step 4: Choose Metrics for Evaluating Performance
Basically if we're able to select the poor waves or not and the final dataset does not have the really poor waves.

***

### Step 5: Generating Simulated Data
Start by generating each set of data from above.

In [400]:
import numpy as np
# Fix random seed
initseed = 123456789
np.random.seed(initseed)

In [401]:
numvals = 1000
# First, build the relevant linspace to grab 1000 points
times = np.linspace(0, 1, numvals)
# Then define the general sine wave used throughout
sin = np.sin(2 * np.pi * times)
# Define function for white noise
def gen_whitenoise(mean, std, size, seed, initseed):
    np.random.seed(seed)
    retval = np.random.normal(mean, std, size=size)
    np.random.seed(initseed)
    return retval

In [402]:
# Success Case 1 Data
# 5 of the same sine waves
success1 = np.column_stack([sin] * 5)
#print success1.shape

In [403]:
# Success Case 2 Data
# 5 same sine waves, 1 with white noise
success2 = np.column_stack([sin] * 5)
wn = gen_whitenoise(0, 6, numvals, 0, initseed)
success2[:, 4] = success2[:, 4] + wn.T
#print success2[0]

In [404]:
# Success Case 3 Data
# 5 different amplitude sine waves, 1 with white noise
success3 = np.column_stack([sin] +
                           [sin * 2] +
                           [sin * 3] +
                           [sin * 4] +
                           [sin * 5])
success3[:, 4] = success3[:, 4] + wn.T
#print success3[0]
#print success3[1]

In [405]:
# Hopeful Case 1 Data
# 10 sine waves, 8 each with small amounts of white noise, 2 with a lot
hope1 = np.column_stack([sin] * 10)
for i in range(0, 8):
    hope1[:, i] += gen_whitenoise(0, 0.25, numvals, i, initseed)
for i in range (8, 10):
    hope1[:, i] += gen_whitenoise(0, 2, numvals, i, initseed)
    
#print hope1[:, 0]

In [406]:
# Fail Case 1 Data
# 5 same sine waves, 4 with white noise
fail1 = np.column_stack([sin] * 5)
for i in range(0, 4):
    fail1[:, i] = fail1[:, i] + gen_whitenoise(0, 2, numvals, i, initseed)
    
#print fail1[0]

***
### Step 6: Plot Simulated Data

In [407]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
from plotly.graph_objs import *
from plotly import tools
init_notebook_mode()

In [408]:
# Success 1

# Setup plotly data
datasets =[]

for i in range(0, 5):
    datasets.append(Scatter(
        x = times,
        y = success1[:,i],
        name = 'Sample ' + str(i)
    ))

data = [datasets[:]]

# Setup layout

layout = dict(title = 'Success 1',
              xaxis = dict(title = 'Time'),
              yaxis = dict(title = 'Unit'),
              )

# Make figure object

fig = dict(data=datasets, layout=layout)

iplot(fig)

In [409]:
# Success 2

# Setup plotly data
datasets =[]

for i in range(0, 5):
    datasets.append(Scatter(
        x = times,
        y = success2[:,i],
        name = 'Sample ' + str(i)
    ))

data = [datasets[:]]

# Setup layout

layout = dict(title = 'Success 2',
              xaxis = dict(title = 'Time'),
              yaxis = dict(title = 'Unit'),
              )

# Make figure object

fig = dict(data=datasets, layout=layout)

iplot(fig)

In [410]:
# Success 3

# Setup plotly data
datasets =[]

for i in range(0, 5):
    datasets.append(Scatter(
        x = times,
        y = success3[:,i],
        name = 'Sample ' + str(i)
    ))

data = [datasets[:]]

# Setup layout

layout = dict(title = 'Success 3',
              xaxis = dict(title = 'Time'),
              yaxis = dict(title = 'Unit'),
              )

# Make figure object

fig = dict(data=datasets, layout=layout)

iplot(fig)

In [411]:
# Hopeful 1

# Setup plotly data
datasets =[]

for i in range(0, 10):
    datasets.append(Scatter(
        x = times,
        y = hope1[:,i],
        name = 'Sample ' + str(i)
    ))

data = [datasets[:]]

# Setup layout

layout = dict(title = 'Hope 1',
              xaxis = dict(title = 'Time'),
              yaxis = dict(title = 'Unit'),
              )

# Make figure object

fig = dict(data=datasets, layout=layout)

iplot(fig)

In [412]:
# Fail 1

# Setup plotly data
datasets =[]

for i in range(0, 5):
    datasets.append(Scatter(
        x = times,
        y = fail1[:,i],
        name = 'Sample ' + str(i)
    ))

data = [datasets[:]]

# Setup layout

layout = dict(title = 'Fail 1',
              xaxis = dict(title = 'Time'),
              yaxis = dict(title = 'Unit'),
              )

# Make figure object

fig = dict(data=datasets, layout=layout)

iplot(fig)

***

### Step 7: Write Algorithm Code

**Go step by step.**

**Reshape Data!**

In [413]:
# Reshape Data: Run with no reshaping

electrodes = 0
trials = 0
times = 0

print len(success1.shape)

def reshape(inEEG):
    if len(inEEG.shape) == 3:
        electrodes = inEEG.shape[1]
        times = inEEG.shape[0]
        trials = inEEG.shape[2]
        return np.reshape(inEEG, (inEEG.shape[0] * inEEG.shape[2], inEEG.shape[1]))
    elif len(inEEG.shape) != 1 and len(inEEG.shape) != 2:
        # fail case
        print "fail"
    else:
        return inEEG
    
print "Final dimensions:", reshape(success1).shape

2
Final dimensions: (1000L, 5L)


In [414]:
# Reshape Data: Run with reshaping

success1 = np.expand_dims(success1, 2)
print len(success1.shape)
print success1.shape
print "Final dimensions:", reshape(success1).shape
success1 = reshape(success1)

3
(1000L, 5L, 1L)
Final dimensions: (1000L, 5L)


In [415]:
# Reshape Data: run with data where reshaping actually matters
dummy = np.dstack([success1] * 2)
print dummy.shape
print dummy[:,:,1].shape
print "Final dimensions:", reshape(dummy).shape

(1000L, 5L, 2L)
(1000L, 5L)
Final dimensions: (2000L, 5L)


**Kernel density probability dist**

In [416]:
# Now define the probability density functions tested in other notebooks
from sklearn.neighbors.kde import KernelDensity
# Run the KDE!
def gaus_kde(indata, bw):
    kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(indata[:, np.newaxis])
    return kde.score_samples(indata[:, np.newaxis])

def hat_kde(indata, bw):
    return KernelDensity(kernel='tophat', bandwidth=bw).fit(indata[:, np.newaxis])
    return kde.score_samples(indata[:, np.newaxis])

def epan_kde(indata, bw):
    return KernelDensity(kernel='epanechnikov', bandwidth=bw).fit(indata[:, np.newaxis])
    return kde.score_samples(indata[:, np.newaxis])

**Define what's inside loop the KDE will execute in**

In [417]:
regelec = success1[:, 0]
#print regelec
probdist = gaus_kde(regelec, .5)
#print probdist
# get joint prob with assumption in log-likelihood format
# leave in log-likelihood to prevent underflow
jointprob = np.sum(probdist)
#print jointprob

**These are really all the modular testable parts of the code. Now will combine into algorithm + ending section**
#### Algorithm Definition:

In [418]:
def prob_baddetec(inEEG, threshold, probfunc):
    electrodes = inEEG.shape[1]
    
    # Start by reshaping data (if necessary)
    if len(inEEG.shape) == 3:
        inEEG = np.reshape(inEEG, (inEEG.shape[0] * inEEG.shape[2], inEEG.shape[1]))
    elif len(inEEG.shape) != 1 and len(inEEG.shape) != 2:
        # fail case
        return -1
    
    # Then, initialize a probability vector of electrode length
    probvec = np.zeros(electrodes)
    
    # iterate through electrodes and get joint probs
    for i in range(0, electrodes):
        # get prob distribution
        probdist = probfunc(inEEG[:, i], .75)
        # using probdist find joint prob
        probvec[i] = np.sum(probdist) 
    
    # normalize probvec
    # first calc mean
    avg = np.mean(probvec)
    # then st, d dev
    stddev = np.std(probvec)
    # then figure out which electrodes are bad
    badelec = []
    print probvec
    for i in range(0, len(probvec)):
        print i, (avg - probvec[i]) / stddev
        if ((avg - probvec[i]) / stddev) >= threshold:
            badelec.append(i)
            
    return badelec

In [419]:
def good_elec(inEEG, badelec):
    for elec in sorted(badelec, reverse = True):
        del inEEG[badelec]
    return inEEG

***

### Step 8: Write Qualitative Evaluation Code
Simply replot signals minus the bad electrodes.

In [420]:
def qual_eval(badelec, expected):
    print "Expected:", expected
    print "Actual:", badelec

def qual_plot(data, title):
    # Setup plotly data
    datasets =[]
    for i in range(0, data.shape[1]):
        datasets.append(Scatter(
            x = times,
            y = data[:,i],
            name = 'Sample ' + str(i)
        ))

    data = [datasets[:]]

    # Setup layout

    layout = dict(title = title,
                  xaxis = dict(title = 'Time'),
                  yaxis = dict(title = 'Unit'),
                  )

    # Make figure object

    fig = dict(data=datasets, layout=layout)

    iplot(fig)

***
### Step 9: Write Quantitative Evaluation Code
Just check the correct number of bad electrodes were removed and which ones were removed

In [421]:
def quant_eval(badelec, expected):
    if set(x) == set(y):
        return true
    else:
        return false


***
### Step 10: Run Qualitative Code

In [422]:
s1bad = prob_baddetec(success2, 3, gaus_kde)

[-1186.62821323 -1186.62821323 -1186.62821323 -1186.62821323 -3190.446102  ]
0 -0.5
1 -0.5
2 -0.5
3 -0.5
4 2.0


In [423]:
# First run on all datasets
s1bad = prob_baddetec(success1, 3, gaus_kde)
s2bad = prob_baddetec(success2, 3, gaus_kde)
s3bad = prob_baddetec(success3, 3, gaus_kde)
h1bad = prob_baddetec(hope1, 3, gaus_kde)
f1bad = prob_baddetec(fail1, 3, gaus_kde)

[-1186.62821323 -1186.62821323 -1186.62821323 -1186.62821323 -1186.62821323]
0 nan
1 nan
2 nan
3 nan
4 nan
[-1186.62821323 -1186.62821323 -1186.62821323 -1186.62821323 -3190.446102  ]
0 -0.5
1 -0.5
2 -0.5
3 -0.5
4 2.0
[-1186.62821323 -1673.76405161 -1982.90114435 -2212.91028122 -3340.88699287]
0 -1.24284986286
1 -0.564709693445
2 -0.134360981798
3 0.185833967444
4 1.75608657066
[-1231.18338155 -1233.17480527 -1238.32873989 -1233.58231129 -1233.92617209
 -1227.49412306 -1234.36024107 -1223.46544724 -2179.59412284 -2189.26531558]
0 -0.501950827541
1 -0.496724289294
2 -0.483197667087
3 -0.495654780195
4 -0.494752309485
5 -0.511633372924
6 -0.49361308526
7 -0.52220672704
8 1.98717539339
9 2.01255766544
[-2163.59651709 -2161.19518534 -2184.01050543 -2182.64371227 -1186.62821323]
0 0.47637901072
1 0.470293609305
2 0.528111685043
3 0.524647988285
4 -1.99943229335


In [424]:
qual_eval(s1bad, [])
qual_plot(good_elec(success1, s1bad), "Success 1")

Expected: []
Actual: []


In [425]:
qual_eval(s2bad, [4])
qual_plot(good_elec(success2, s2bad), "Success 2")

Expected: [4]
Actual: []


In [426]:
qual_eval(s3bad, [4])
qual_plot(good_elec(success3, s3bad), "Success 3")

Expected: [4]
Actual: []


In [427]:
qual_eval(h1bad, [8, 9])
qual_plot(good_elec(hope1, h1bad), "Hope 1")

Expected: [8, 9]
Actual: []


In [428]:
qual_eval(f1bad, [])
qual_plot(good_elec(fail1, f1bad), "Fail 1")

Expected: []
Actual: []


***
### Step 11: Run Quantitative Code

Honestly, anything being gotten out of quanitative eval was gotten earlier

***
### Step 12: Anlayzing Evaluations

Unfortunately, what I expected to happen didn't exactly happen. At 3 standard deviations away (which is a standard lent by what the researchers used in their own probability implementation), none of the noisy data was there. Attempting to modify the noise inputted into the data by changing the scale of which the noise was being added (ie changing stddev from 1 -> 2 -> 3) had literally no effect on the outcome. What makes sense is that maybe because the number of white noise values being used is so high, they accurately depict a normal graph, and if they depict a normal graph, the distribution that is eventually modelled by the kernel probability is essentially a normal graph. Not sure best way to deal with this problem (or should I fix simulations?) but will ask Jovo for help before moving more